# BlockEncoding class 101

In classical computer science, we often perform operations directly on matrices, such as inversion, exponentiation, or polynomial transformations. In quantum computing, however, all physical operations must be unitary (represented by matrices U such that $U^\dagger U=\mathbb{1}$), which means they are reversible and preserve the norm of the state vector.

Most matrices of interest, such as Hamiltonians (which describe system energy), are Hermitian but not unitary. To solve this, modern quantum algorithms use Block Encodings to "embed" these non-unitary matrices into a larger unitary system, and then use Quantum Signal Processing (QSP) and its generalizations to manipulate their values. 

## What are block-encodings?
Block encoding is a foundational technique that allows a quantum computer to represent a general matrix $A$ (where $∥A∥\leq1$) as the top-left block of a larger unitary matrix $U$.

We say that a unitary $U$ is an $(\alpha,m,\epsilon)$-block-encoding of $A$ if, where $\alpha$ is a scaling factor, $m$ is the number of ancillary qubits, and $\epsilon$ is the precision. We essentially put a (possibly) non-unitary matrix in the top left block of a larger, unitary matrix. By projecting onto that top left subspace, we then essentially act exactly with the part of the matrix we wanted to apply, with the rest elements being disregarded.
$$\begin{pmatrix}
\frac{A}{\alpha} & *\\
* & *
\end{pmatrix}$$

In Qrisp, we can construct a block encoding in three ways:

- The ``.from_array`` constructs a BlockEncoding from a numpy array,

- ``.from_operator`` constructs a Blockencoding directly from a Hamiltonian constructed via QubitOperators, or FermionicOperators,

- and lastly, the ``.from_lcu`` constructs a BlockEncoding using the Linear Combination of Unitaries approach.

The first two options are rather self-explanatory, really... You (the user) provide the matrix and/or Hamiltonian you'd like to block encode, invoke the appropriate method for your desired input and, whoosh, Qrisp block encodes it for ya.




In [ ]:
import numpy as np
from qrisp.block_encodings import BlockEncoding
A = np.array([[0,1,0,1],
              [1,0,0,0],
              [0,0,1,0],
              [1,0,0,0]])
B_A = BlockEncoding.from_array(A)

from qrisp.operators import X, Y
H = X(0)*X(1) + 0.2*Y(0)*Y(1)
B_H = BlockEncoding.from_operator(H)

Essentially the third approach, ``.from_lcu`` is the epitome of block encoding. I mean, what it does is actually in the acronym: Linear Combination of Unitaries (or for the academic practitioners among you: $A=\sum_i\alpha_i U_i$).

If such a matrix $A$ is expressed as shown above (as a weighted sum of unitaries), the LCU algorithmic primitive uses three (well, technically two) oracles:

- PREPARE prepares (kinda obvious, huh?) a superposition of index states weighted of the square roots of the coefficients $\alpha_i$ in the ancillary variable:$$\text{PREP}\ket{0}_a=\sum_{i=0}^{m-1}\sqrt{\frac{\alpha_i}{\alpha}}\ket{i}_a$$

- SELECT applies the corresponding unitary $U_i$ to the operand (target) variable, controlled by the ancillary variable:$$\text{SEL}=\sum_{i=0}^{m-1}\ket{i}\bra{i}_a\otimes U_i$$,

- and finally PREPARE$^\dagger$, which then essentially unprepares the ancillary variable.

We have successfully applied $A$ to our operand when the ancillary variable is measured in the $\ket{0}$ state.

For the visual learners, here is a figure that will (hopefully) tie everything you read so far together.

![Alt text](BE_bad_select.png)

We could also define a block encoding as a pair of unitaries $(U, G)$ ($G$ is another name for PREPARE, $U$ is another name for SELECT). Their action on quantum states can be captured by the following expressions (don't worry, we'll also use words about what this means immediately after).

$G$ simply prepares the ancilary variable in a superposition defined by the coefficients (or the weights) of the weighted sum of unitaries (LCU):$$G\ket{0}_{in\_case}=\ket{G}_{in\_case}.$$ Here we've denoted the ancillary variable as ``in_case``, short for "inner case".

$U$, on the other hand then selects "just the right amount" of each of the underlying unitaries in the weighted sum: $$U\ket{i}_{in\_case}\ket{\psi}_{operand}=\ket{i}_{in\_case}U_i\ket{\psi}_{operand}$$.

As seen in the figure above, this can be done via multiple controlled operations. In Qrisp, however, we use ``qswitch``, which is just another name for the select operator (multiplexor is another name found in literature for the same thing, btw). What's so special about ``qswitch``? Well, we've implemented a massively more efficient approach about how to apply such a (quantum) switch case (hence the name) based on balanced binary trees. For further information about this, please refer to the paper from Khattar and Gidney, [Rise of conditionally clean ancillae for efficient quantum circuit constructions](https://arxiv.org/pdf/2407.17966v1).

How are we feeling? I assume that the researchers in the field could be portrayed with this emoji (😎), while people coming from outside of quantum can feel a bit overwhelmed (🥲). Let's balance the target audiences by a relevant example that is the cornerstone of solving partial differential equations, where we also explain how to use the ``.from_lcu`` method (went quite the tangent, huh? Glad you're still here - it'll be worth it!).

### Custom BlockEncoding - the discrete Laplacian

In this example, we construct a block encoding of the 1D discrete Laplacian operator with periodic boundary conditions using the ``.from_lcu`` method. The discrete Laplace operator is central in numerical physics and differential equations, representing the second derivative on a grid. 

Let's start by constructing the matrix in Numpy and print it so that everyone is on the same page about what we'll be doing in this example.

In [ ]:
import numpy as np

N = 8
I = np.eye(N)
L = 2 * I - np.eye(N, k=1) - np.eye(N, k=-1)
L[0, N-1] = -1
L[N-1, 0] = -1

print(L)

This is exactly the finite-difference Laplacian with wrap-around endpoints.

We can, of course, use the ``.from_array`` method you're already a master of. Actually, let's do that real quick:

In [ ]:
B_L = BlockEncoding.from_array(L)

Why even bother with ``.from_lcu`` then? Fair question. The answer hides in what's happening underneath, and the amount of **quantum resources** that these two methods use.

## Quantum Resource Estimation w/ ``.resources``

"Whoa, whoa, whoa?! Quantum resources? As in quanutm resources in Quantum Resource Estimation?" you ask.

"Yes.", we answer.

"But isn't it currently a research endeavor about finding these exact resource estimates?!"

"Yes. But... with Qrisp we made it as simple, clean, and qrispy (had to go for it), as calling the ``.resources`` method."

Let's see what do we get as an output:

In [ ]:
from qrisp import QuantumFloat

def operand_prep():
    return QuantumFloat(2)
quantum_resources = B_L.resources(operand_prep)()
print(quantum_resources)

Alright, it's a dictionary with some letters (or a string of them), and a number. The former is the detailed **gate count**, or rather the number of every specific gate used in the application of the block encoding using ``.from_array``. The second number is the **depth** of the circuit (for lack of a better word - we're trying to eliminate this sacrilegious word in qrisp), which is related to how long the calculation will take on an actual quantum computer. The longer it takes, the more likely that the qubits decohere, resulting in noisy results that tell us nothing.

Let's get back on track. We've block encoded the Laplacian and gathered the resources... Let's now do the exact same thing, but more efficiently by constructing a custom block encoding with ``.from_lcu``.

The Laplacian can be decomposed as $\Delta = 2\mathbb{1}-V-V^\dagger$, where 

- $\mathbb{1}$ is the identity, 

- $V$ is the forward shift operator: $V\ket{k}=-\ket{k+1 \mod N}$, and 

- $V^\dagger$ is the backward shift $V^\dagger\ket{k}=-\ket{k-1\mod N}$.

The negative sign ensures Hermiticity and matches the chosen finite-difference convention.

This decomposition fits naturally into the LCU framework:$$\Delta=\sum_{i=0}^2\alpha_i U_i\propto 2\mathbb{1}+V+V^\dagger$$

This can be easily implemented in Qrisp with the built-in arithmetic it allows:

In [ ]:
from qrisp import gphase

def I(qv):
    # Identity: do nothing
    pass

def V(qv):
    # Forward cyclic shift with a global phase -1
    qv += 1
    gphase(np.pi, qv[0])  # multiply by -1

def V_dg(qv):
    # Backward cyclic shift with a global phase -1
    qv -= 1
    gphase(np.pi, qv[0])

Now combine them as a linear combination of unitaries with coefficients [2, 1, 1]:

In [ ]:
unitaries = [I, V, V_dg]
coeffs = np.array([2.0, 1.0, 1.0])

And thats it... that's all the ingredients we need to define a custom block encoding with ``.from_lcu``. We just provide the coefficients and the unitaries, and the BlockEncoding class does all the management of the coefficients, ancillary qubits, and whatever else under the sun. In future releases we also want it to be able do perform ones taxes.

All jokes aside, let's block encode this Laplacian and check the resources if it's actually more efficient (it is) before explaining why it's more efficient.

In [ ]:
BE = BlockEncoding.from_lcu(coeffs, unitaries)
quantum_resources = BE.resources(operand_prep)()
print(quantum_resources)

Let's now compare the outputs.

{'gate counts': {'cy': 4, 'u3': 14, 'gphase': 2, 't_dg': 28, 'x': 9, 'cx': 67, 't': 21, 'h': 14, 'p': 4}, 'depth': 107}

{'gate counts': {'u3': 6, 'gphase': 2, 't_dg': 12, 'x': 5, 'cx': 36, 't': 10, 's': 2, 'h': 10, 'p': 2, 'measure': 2}, 'depth': 58}

Even if you don't know what any of the letters or strings actually mean, you already get the idea, that the values of the bottom dictionary are lower than the one on the top. To say it in quantum jargon, the ``.from_lcu`` is (much) more efficient than ``.from_array``, which is just a simpler interface to ``.from_operator``. 

Why is that? Well, it all comes down to which unitaries and how many do we want to combine to construct the block encoding. In the custom block encoding it's rather self apparent: we need three unitaries to construct the Laplacian, as seen above. The ``.from_array`` and ``.from_operator`` methods, on the other hand, construct block encodings by decomposing the Hamiltonian into a linear combination of Pauli strings. If you're asking what is a Pauli string, here's a quick intuitive answer.

Think of a Pauli string as a multi-qubit instruction. It is a sequence of Pauli operators (matrices) $I$, $X=\sigma_X$, $Y=\sigma_Y$, and $Z=\sigma_Z$. Each individual Pauli matrix is a simple $2 \times 2$ unitary matrix. When you have multiple qubits, you take the tensor product of these matrices. For example, a Pauli string for a 3-qubit system might look like $X \otimes Z \otimes I$ (often shortened to $XZI$). Because Pauli strings are both unitary and Hermitian, they are the "gold standard" for quantum hardware. If you can break an operator down into Pauli strings, a quantum computer knows exactly how to execute it.

And this is exactly what the methods do. It takes an arbitrary square matrix and expresses it as a weighted sum of these Pauli strings ($A=\sum_j\alpha_j P_j$), where $\alpha_j$ is a scalar coefficient (aka a number) and $P_j$ is a Pauli string.

When you use a method like ``.from_array``, the algorithm has no "intuition" about the structure of your matrix. It performs a systematic projection of your matrix onto every possible Pauli string. For an $n$-qubit system, there are $4^n$ possible Pauli strings. If your matrix doesn't happen to align perfectly with the Pauli basis, the decomposition results in a massive list of strings, many with tiny coefficients. 

In the case of our 1D Laplacian, Pauli Decomposition might require up to 16 different Pauli strings to represent the matrix. LCU, on the other hand, requires only 3 custom unitaries (like the ones we manually defined) to represent the exact same matrix.

Building a block encoding requires a number of "ancilla" qubits and gate operations proportional to the number of terms in your sum. The more terms, the more complex the circuit, leading to more noise, requiring more qubits. 

Having covered the different approaches to constructing block encoding and analysing their quantum resource requirements already sounds extremely useful for (at least) the academics among you who are working on or developing new ways to block encode certain matrices. In that case, performing resource estimation on your approach is incredibly simple (calling ``.resources``) after the approach is implemented in Qrisp. If you'd like some help to implementing your approach, please reach out so that we can include it in future releases. The idea is to have many different methods implemented at one place so that the user can then always use the most efficient one for their problem based on the quantum resources they have at their disposal.

Let's show how we can obtain the state we've successfully applied LCU onto, or run the simulation with the Repeat-Until-Success protocol.

## Applying BlockEncodings w/  ``.apply`` & ``.apply_rus``

Now that we’ve built our block encoding, we face the ultimate "So what?" moment. A block encoding is like having a high-performance engine sitting in your garage. It's impressive to look at, but eventually, you actually want to drive it. In our case, "driving" means applying that matrix to a quantum state.

This is where ``.apply`` and ``.apply_rus`` come into play.

### The manual, NISQy way: ``.apply``
The ``.apply`` method is the straightforward way to use your block encoding. It simply adds the necessary gates to your circuit to perform the unitary $U$.However, there’s a catch. Because we are block encoding a non-unitary matrix $A$ inside a larger unitary $U$, the "math" only works out perfectly when our extra qubits (the ancillas) start in the $\ket{0}$ state and end in the $\ket{0}$ state. When you call .apply(qv), Qrisp returns a list of these ancilla qubits. To get the correct result, you have to perform post-selection. This is a fancy way of saying: "Run the experiment, measure the ancillas, and if they aren't all zero, throw that result in the trash and try again."As you can imagine, if your success probability is low, you'll be throwing away a lot of data. It’s effective, but about as efficient as trying to win the lottery by buying one ticket at a time and waiting a week for the results.

Since current quantum hardware often struggles with mid-circuit measurements and real-time feedback (the features required for RUS), we frequently have to fall back on post-selection. Essentially you run the experiment many times, but you only keep the actors who followed the script perfectly (i.e., the ancillas stayed in $\ket{0}$). 

Here is how you implement this for a Heisenberg Hamiltonian using a standard simulator (or your favorite NISQ hardware) backend.

In [ ]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y, Z

H = sum(X(i)*X(i+1) + Y(i)*Y(i+1) + Z(i)*Z(i+1) for i in range(3))
BE = BlockEncoding.from_operator(H)
b = np.array([1.,2.,3.,1.,1.,2.,3.,1.,1.,2.,3.,1.,1.,2.,3.,1.])

# Prepare initial system state
operand = QuantumFloat(4)
prepare(operand, b)

# Apply the operator to an initial system state
ancillas = BE.apply(operand)

Utilize the Qrisp Backend Interface to define a backend. While this example uses the Qiskit Aer simulator, the interface also supports physical quantum backends.

In [ ]:
from qrisp.interface import QiskitBackend
from qiskit_aer import AerSimulator
example_backend = QiskitBackend(backend = AerSimulator())

# Use backend keyword to specify quantum backend
res_dict = multi_measurement([operand] + ancillas,
                            shots=1000,
                            backend=example_backend)

# Post-selection on ancillas being in |0> state
filtered_dict = {k[0]: p for k, p in res_dict.items() \
                if all(x == 0 for x in k[1:])}
success_prob = sum(filtered_dict.values())
filtered_dict = {k: p / success_prob for k, p in filtered_dict.items()}
amps = np.sqrt([filtered_dict.get(i,0) for i in range(16)])
print(amps)

Alright, we have obtained something. Let's further figure out what that something actually is. Even before that, let's run a classical comparison that will help us make sense of the output. Let us quickly perform some magic, what's important here is the output, which is the classically obtained result for the same problem we've solved quantumly using ``.apply`` and post-selection.

In [ ]:
H_arr = H.to_array()

psi = H_arr @ b
psi = psi / np.linalg.norm(psi)
print(psi)

print(np.linalg.norm(psi-amps))

If we again compare the two, as well as the variance, we see that this fluctualtes by quite a lot. It is becasue of the probabilistic nature of quantum computation - if we would increase the number of shots, the variance would decrease, and the results won't fluctuate that much.

This is the NISQ, post-selection based to solve this. We designed Qrisp as being forward thinking with a lot of the infrastructure required for the Fault Tolerance era already introduced, like real-time measurements and repeat until success procedure. Let's show how to use the latter with our BlockEncodings, and see how the variance of the results looks like then.

### The automatic, FT way: ``.apply_rus``

If you don't feel like manually filtering your data or wasting quantum shots, you use ``.apply_rus``. The RUS stands for Repeat-Until-Success.

Instead of you manually checking the ancillas, the ``apply_rus`` method wraps the entire process into a loop:

- It prepares your input state.

- It applies the block encoding.

- It measures the ancillas.

- If the ancillas are not zero, it resets them and starts over automatically.

- If they are zero, it stops. You now have the exact state you wanted. If not, the loop repeats again.

This protocol turns a probabilistic process into a deterministic one. From your perspective as a coder, it looks like the matrix was applied successfully 100% of the time. Let's show that with the same example from above.

In [ ]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y, Z

H = sum(X(i)*X(i+1) + Y(i)*Y(i+1) + Z(i)*Z(i+1) for i in range(3))
BE = BlockEncoding.from_operator(H)

# Prepare initial system state
def operand_prep():
    operand = QuantumFloat(4)
    b = np.array([1.,2.,3.,1.,1.,2.,3.,1.,1.,2.,3.,1.,1.,2.,3.,1.])
    prepare(operand, b)
    return operand

# Apply the operator to an initial system state
@terminal_sampling
def main():
    return BE.apply_rus(operand_prep)()
res_dict = main()

amps = np.sqrt([res_dict.get(i,0) for i in range(16)])

H_arr = H.to_array()

psi = H_arr @ b
psi = psi / np.linalg.norm(psi)
print(psi)
print(np.linalg.norm(psi-amps))

A few things to note here. We pass the operand (the state we have prepared and want to apply the LCU protocol to) as a function, as oppossed as a QuantumVariable. The second difference is the way we obtain the BlockEncoding and apply the ``.apply_rus``, which is within a main function. This is because we use the Jasp compilation pipeline, which allows us to avoid the pythonic compilation bottleneck. You can learn more bout it in the Jasp tutorial.

By invoking the @terminal_sampling decorator, we sample this probability plenty of times. 

In the end, we obtain a result that is consistent when repeating the simulation. This also observed in the variance. We essentially obtain the correct solution all the time with the variance not based on the amount of shots. How neat is that?!

Before delving into the final section of this tutorial, let's quickly recap. The ``.apply`` method is great if you are doing complex manual circuit manipulations or if you’re performing resource estimation and don't want the overhead of the RUS loop logic.

But for most "real-world" (or real-simulation) use cases, ``.apply_rus`` is your best friend. It takes a function that prepares your qubits (operand_prep) and returns a new function that handles the messy "measure-reset-repeat" logic for you.

Oh, and you can perform quantum resource estimation even before simulating. How nice, right?!

## Expressing new BlockEncodings using arithmetic operations

Up until now, we’ve treated Block Encodings as static objects—you build them, and you apply them. But in the real world (and especially in quantum chemistry or condensed matter physics), operators are rarely solitary. They are sums of interactions, products of symmetries, and scaled potentials.

To handle this, Qrisp allows you to perform a full suite of algebraic operations directly on your BlockEncoding objects:

- Addition (``+``): Implements $A + B$ via the LCU framework.

- Subtraction (``-``): Implements $A - B$.

- Scalar Multiplication (``*``): Scales an operator by a constant $c \cdot A$.

- Negation (``-A``): Flips the sign of the operator.

- Matrix Multiplication (``@``): Composes two operators $A \cdot B$.

- Kronecker Product (``.kron()``): Performs the tensor product $A \otimes B$ to expand the Hilbert space.

We will expand this list in future releases, as well as implement more efficient ways for some of these operations. For those curious, we took inspiration from the paper [Products between block-encodings](https://arxiv.org/pdf/2509.15779) to implement the underlying logic.

Why and how would this be useful, you might be wondering. Well, if you had to manually track the coefficients, normalization factors ($\alpha$), and ancilla registers every time you wanted to add two matrices together, you’d probably go back to classical computing by lunchtime. This is where the BlockEncoding arithmetic comes in. In Qrisp, we treat BlockEncoding objects as first-class programming abstractions.

Think of it like this: When you write ``x = 1.5 * y + 2.0``, you don't care how the CPU handles floating-point registers or carry bits. You care about the relationship between ``x`` and ``y``.

By using standard Python operators (``+``, ``-``, ``*``, ``@``) on BlockEncoding objects, you are:

- Automating LCU: Addition and subtraction automatically trigger the Linear Combination of Unitaries (LCU) framework behind the scenes.

- Managing Complexity: Qrisp keeps track of the expanding ancilla requirements and the cumulative normalization factor ($\alpha$) so you don't have to.

- Mirroring the Math: Your code starts looking exactly like the physics paper you're trying to implement.

Note of Caution: Just because you can add fifty block encodings together doesn't mean you should. Each arithmetic operation adds circuit depth and ancilla qubits. For complex polynomials, Quantum Signal Processing (QSP) is usually the more efficient path—but (more on that on the other tutorial) for building Hamiltonians, arithmetic is king.

Let's go through some examples!

In [ ]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Z

# Define two simple operators
A = BlockEncoding.from_operator(X(0) * X(1))
B = BlockEncoding.from_operator(Z(0) * Z(1))

# Perform arithmetic: 2.5 * A - B
# Qrisp handles the LCU logic and the Z-gate for the sign flip automatically
H_total = 2.5 * A - B

def prep_zeros():
    return QuantumVariable(2)

@terminal_sampling
def run_arithmetic():
    # Use apply_rus to see the result of the combined operator
    return H_total.apply_rus(prep_zeros)()

print(f"New Alpha: {H_total.alpha}") # Alpha is now 2.5 + 1.0 = 3.5
print(f"Resulting Distribution: {run_arithmetic()}")

In this example, we’ll build a composite operator $H_{total} = 2.5 A - B$. This demonstrates how Qrisp handles both scalar multiplication and the sign flip of subtraction. Let's do another one!

In [ ]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Z

# Create two 2-qubit block encodings
BE1 = BlockEncoding.from_operator(X(0) * X(1))
BE2 = BlockEncoding.from_operator(Z(0) * Z(1))

# 1. Expand the Hilbert space: BE1 (on qubits 0,1) tensor BE2 (on qubits 2,3)
# The resulting BE acts on 4 qubits total.
BE_large = BE1.kron(BE2)

# 2. Composition: Multiply the large system by itself (A^2)
# The '@' operator implements operator multiplication (matrix product)
BE_squared = BE_large @ BE_large

def operand_prep():
    return QuantumVariable(4)

@terminal_sampling
def main():
    return BE_squared.apply_rus(operand_prep)()

print(f"Total Ancillas: {len(BE_squared._anc_templates)}")
print(f"Final State: {main()}")

Sometimes you want to build a large system by gluing smaller ones together. Here, we use ``.kron()`` to combine two 2-qubit systems into a 4-qubit system, and then use ``@`` to compose it with another operator.

## Recap

We have journeyed from the mathematical definition of Block Encodings to high-level arithmetic operations on quantum operators. Here is the distilled wisdom from this tutorial:

- The Bridge to Unitary: Block Encoding is the essential technique that allows us to run non-unitary math (like Hamiltonians or Laplacians) on unitary quantum hardware by embedding them into larger spaces.

- Flexibility in Construction: Qrisp meets you where you are. Use ``.from_array`` for quick prototyping, ``.from_operator`` for chemistry/physics Hamiltonians, or ``.from_lcu`` when you need maximum efficiency and custom unitary definitions.

- Efficiency Matters: As demonstrated with the Laplacian, choosing the right encoding strategy (LCU vs. Pauli decomposition) can drastically reduce circuit depth and gate count. The ``.resources()`` method is your built-in tool to verify this.

- Deterministic Execution: While ``.apply`` works for NISQ experiments via post-selection, ``.apply_rus`` leverages the Repeat-Until-Success protocol to provide deterministic, Fault-Tolerant compatible execution.

- Composability: You can build complex operators using standard Python math (``+``, ``-``, ``@``, ``*``). Qrisp handles the heavy lifting of managing coefficients and ancilla qubits, letting you focus on the algorithm logic.

By mastering Block Encodings in Qrisp, you aren't just manipulating matrices; you are structuring quantum algorithms in a way that is modular, scalable, and ready for the future of fault-tolerant computing. Now, go forth and encode!

## Motivation

MFeeling like you mastered the BlockEncodings class? Good. Now let's use it on some state-of-the-art research applications. In the next tutorial, we move from simple algebra to Quantum Functional Analysis, transforming a matrix’s entire spectrum using Quantum Signal Processing (QSP).

Standard matrix multiplication is expensive. Qubitization bypasses this by using Chebyshev polynomials $T_n(A)$ to "shape" your matrix’s eigenvalues with near-zero overhead.

The logic is elegant: by interleaving your block encoding with specific phase shifts, you can apply any polynomial $P(A)$ to your matrix. While finding these "phase angles" is usually a classical nightmare, Qrisp calculates them for you internally.

We’ve distilled these world-class techniques into three simple methods:

- ``.poly``: Apply custom polynomial transformations.

- ``.inv``: Perform matrix inversion without the LCU headache.

- ``.sim``: Execute Hamiltonian simulation with optimal scaling.

Ready to see the magic? Journey onwards to the next tutorial to continue learning about quantum linear algebra and how Qrisp's BlockEncoding interface makes it simple and intuitive!